In [ ]:
! git clone https://github.com/sakethc4/FIRE-Research-Matrix-Multiplication
! cd FIRE-Research-Matrix-Multiplication/

fatal: destination path 'FIRE-Research-Matrix-Multiplication' already exists and is not an empty directory.


In [ ]:
!pip install pennylane==0.33.1 pennylane-qiskit==0.33.1 memory-profiler line-profiler

In [ ]:
from memory_profiler import profile
import pennylane as qml
import pennylane.numpy as np
import time
from functools import wraps
from line_profiler import LineProfiler
import csv

# Defines quantum device
# wires_m represents the multiplicand wires
# wires_solution represents the solution for quantum fourier transforms
wires_m=[0,1,2,3,4,5]
wires_solution=[6,7,8,9,10,11,12,13,14,15,16]

# initialize quantum device
dev = qml.device("default.qubit", wires=wires_m  + wires_solution, shots=1)

n_wires = len(dev.wires)

add_time=0
mul_time=0
total_time=0

# Decorator used to track the execution time of addition operations
def calculate_time_add(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time() # Record start time
        result = func(*args, **kwargs) # Call function
        end = time.time()
        execution_time = (end - start) * 1000  # Time to milliseconds
        global add_time
        add_time+=execution_time
        return result
    return wrapper

# Decorator used to track the execution time of multiplication operations
def calculate_time_mul(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time() # Record start time
        result = func(*args, **kwargs) # Call function
        end = time.time()
        execution_time = (end - start) * 1000  # Time to milliseconds
        global mul_time
        mul_time+=execution_time
        return result
    return wrapper

# Decorator used to track the execution time of matrix multiplication operations
def calculate_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time() # Record start time
        result = func(*args, **kwargs) # Call function
        end = time.time()
        execution_time = (end - start) * 1000  # Time to milliseconds
        global total_time
        total_time=execution_time
        return result
    return wrapper

# Function that performs quantum fourier addition on a quantum circuit
def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])

# Function that performs multiplication using quantum fourier trarnsforms
def multiplication(k,wires_m, wires_solution):
    qml.QFT(wires=wires_solution)
    for i in range(len(wires_m)-1,-1,-1):
        qml.ctrl(add_k_fourier, control=wires_m[i])(k, wires_solution[len(wires_m)-1-i:])
    qml.adjoint(qml.QFT)(wires=wires_solution)

# Quantum node for multiplication
@calculate_time_mul
@qml.qnode(dev)
def mul(m,k):
    qml.BasisEmbedding(m, wires=wires_m)  # m encoding
    multiplication(k,wires_m, wires_solution)
    return qml.sample(wires=wires_solution)

# Quantum node for multiplication with sign correction
def mul2(m,k):
    mul_result=mul(abs(m),abs(k))
    mul_result=int(''.join(map(str, list(mul_result))), 2)
    if (m>0 and k>0)|(m<0 and k<0):
        mul_result=mul_result
    else:
        mul_result=-mul_result
    return mul_result

# Quantum node for addition
@calculate_time_add
@qml.qnode(dev)
def sum(m, k):
    qml.BasisEmbedding(m, wires=range(len(wires_solution)))  # m encoding
    qml.QFT(wires=range(len(wires_solution)))  # step 1
    add_k_fourier(k, range(len(wires_solution)))  # step 2
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # step 3
    return qml.sample(wires=range(len(wires_solution)))

# Quantum node for addition that handles oveflow
def sum2(m,k):
    sum_result=sum(m,k)
    sum_result2=int(''.join(map(str, list(sum_result[1:]))), 2)
    if sum_result[0]==1:
        sum_result2=sum(pow(2,n_wires-1),-sum_result2)
        sum_result2=-int(''.join(map(str, list(sum_result2[1:]))), 2)
    return sum_result2

# Adds matricies using quantum fourier transform addition
def Matrix_add(a,b):
    a_order=a.shape[0]
    c=np.zeros((a_order,a_order))
    for i in range(a_order):
        for j in range(a_order):
            c[i][j]=sum2(a[i][j],b[i][j])
            # print(c)
    return c.astype(int)

# Adds 4 numbers using quantum fourier transform addition
@qml.qnode(dev)
def sum_4num(m, k, a, b):
    qml.BasisEmbedding(m, wires=range(len(wires_solution)))  # m encoding
    qml.QFT(wires=range(len(wires_solution)))  # step 1
    add_k_fourier(k, range(len(wires_solution)))  # step 2
    add_k_fourier(a, range(len(wires_solution)))  # step 2
    add_k_fourier(b, range(len(wires_solution)))  # step 2
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # step 3
    return qml.sample(wires=range(len(wires_solution)))

# Adds 4 numbers using quantum fourier transform addition and handles overflow
@calculate_time_add
def sum2_4num(m,k,a,b):
    sum_result=sum_4num(m,k,a,b)
    sum_result2=int(''.join(map(str, list(sum_result[1:]))), 2)
    if sum_result[0]==1:
        sum_result2=sum(pow(2,n_wires-1),-sum_result2)
        sum_result2=-int(''.join(map(str, list(sum_result2[1:]))), 2)
    return sum_result2

# Adds 4 matrices using quantum fourier transforms
def Matrix_add_4num(a,b,c,d):
    a_order=a.shape[0]
    e=np.zeros((a_order,a_order))
    for i in range(a_order):
        for j in range(a_order):
            e[i][j]=sum2_4num(a[i][j],b[i][j],c[i][j],d[i][j])
            # print(c)
    return e.astype(int)

# Split matrix into 4 quadrants to assist strassen(A, B)
def split_matrix(matrix):
    row, col = matrix.shape
    row2, col2 = row // 2, col // 2
    return matrix[:row2, :col2], matrix[:row2, col2:], matrix[row2:, :col2], matrix[row2:, col2:]
@calculate_time

# Completes strassen based multiplication using quantum fourier transfom based operations
def strassen(A, B):
    if min(A.shape) <= 1:
        p_num=mul2(A[0][0],B[0][0])
        p=np.full((1,1),p_num)
        # print(p)
        return p

    # Split matrices into quadrants
    a11, a12, a21, a22 = split_matrix(A)
    b11, b12, b21, b22 = split_matrix(B)

    k1=Matrix_add(a11 , a22)
    k2=Matrix_add(b11 , b22)
    k3=Matrix_add(a21 , a22)
    k4=Matrix_add(b12 , -b22)
    k5=Matrix_add(b21 , -b11)
    k6=Matrix_add(a11 , a12)
    k7=Matrix_add(a21 , -a11)
    k8=Matrix_add(b11 , b12)
    k9=Matrix_add(a12 , -a22)
    k10=Matrix_add(b21 , b22)

    # Strassen's 7 multiplications
    m1 = strassen(k1,k2)
    m2 = strassen(k3, b11)
    m3 = strassen(a11, k4)
    m4 = strassen(a22, k5)
    m5 = strassen(k6, b22)
    m6 = strassen(k7, k8)
    m7 = strassen(k9, k10)

    # Compute the quadrants of the result matrix
    c11 = Matrix_add_4num(m1 , m4 , -m5, m7)
    c12 = Matrix_add(m3 , m5)
    c21 = Matrix_add(m2 , m4)
    c22 = Matrix_add_4num(m1 , -m2 , m3, m6)

    # Concatenate the quadrants to get the result
    return np.vstack((np.hstack((c11, c12)), np.hstack((c21, c22))))
k=2
val=2
A=np.full((k,k),val)
B=np.full((k,k),val)
# A=np.array([[1,2],[3,2]])
# B=np.array([[2,3],[1,2]])

# Multiply using Strassen algorithm
C = strassen(A, B)
print("Matrix A:\n", A)
print("Matrix B:\n", B)
print("Result Matrix C:\n", C)

# Initialize values for test output
def init_values(init_k, init_val):
  global k, val, matrix_a, matrix_b, n, m, matrix_c
  k=init_k
  val=init_val
  matrix_a=np.full((k,k),val)
  matrix_b=np.full((k,k),val)
  n=matrix_a.shape[0]
  m=matrix_b.shape[1]
  matrix_c=np.zeros((n,m))

# Create test output in .csv file
if __name__=='__main__':
  csv_file = open('multication_strassen.csv', 'a', newline='')
  for matrix_dimension in [2, 4, 8, 16]:
    init_values(matrix_dimension, 2)
    print("--------- MATRIX OF SIZE ", k, "---------")
    matrix_c = strassen(matrix_a, matrix_b)
    print(matrix_c)
    writer = csv.writer(csv_file)
    data = [['size', 'value', 'add_time', 'mul_time', 'total_time'], [k, val, add_time, mul_time, total_time]]
    writer.writerows(data)
  csv_file.close()


Matrix A:
 [[2 2]
 [2 2]]
Matrix B:
 [[2 2]
 [2 2]]
Result Matrix C:
 [[8 8]
 [8 8]]
--------- MATRIX OF SIZE  2 ---------
[[8 8]
 [8 8]]
--------- MATRIX OF SIZE  4 ---------
[[16 16 16 16]
 [16 16 16 16]
 [16 16 16 16]
 [16 16 16 16]]
--------- MATRIX OF SIZE  8 ---------
[[32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]
 [32 32 32 32 32 32 32 32]]
--------- MATRIX OF SIZE  16 ---------
[[64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64 64 64 64 64 64 64 64 64 64 64 64 64 64]
 [64 64 64